**<h1> This is my first notebook publish in Kaggle. </h1>**
*<h2>Hope I could add something</h2>*

***

*Please note that this notebook is in Python 2.7. So I dont know how well it will run in online here in Kaggle. However, if you use Python 2.7, you can download the notebook and run it locally in your system -----------*

***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Part 1: Loading and Processing the data
### The data is loaded as a DF and the last column with NaNs are removed

In [ ]:
df = pd.read_csv("../input/data.csv")
df.head()

In [ ]:
df.index = df['id']
df.drop(["id", "Unnamed: 32"], axis = 1, inplace = True)
df.head()

# Part 2: Running basic analysis and Visualizations

### Dividing the dataset into ones which are malignant and ones which are benign and vialulizing all the features
#### To find out which features may be important and which are not

In [ ]:
df.head()

In [ ]:
df.boxplot(by = 'diagnosis', figsize = (20, 20))
plt.show()

## Scaling the data
#### Since the data is highly variable and any feature which has low variance will be neglected, in order to bring them to a level playing field, standard scaling is done.

In [ ]:
from sklearn.preprocessing import StandardScaler as ss
dft = df.drop(['diagnosis'], axis = 1, inplace = False)
scaler = ss().fit(dft)
scaled_df = pd.DataFrame(scaler.transform(dft))

In [ ]:
scaled_df.index = df.index
scaled_df.columns = dft.columns
scaled_df['diagnosis'] = df['diagnosis']
scaled_df.head()

In [ ]:
scaled_df.boxplot(by = "diagnosis", figsize = (20, 20))
plt.show()

### The above boxplots gives us a very simple way of visualizing all th variables at once. 
#### It gives an indication as to which variables might be good for predictions and which wont.

# Part 3: Subsetting the data to training and testing sets

In [ ]:
diagnosis = df.diagnosis
df.drop(['diagnosis'], axis = 1, inplace = True)

In [ ]:
from sklearn.cross_validation import train_test_split as tspl
df_train, df_test, diag_train, diag_test = tspl(df, diagnosis, test_size = 0.33)

# Part 4: Running the KNN Classifier

### K Nearest Neighbour Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
knn = KNN(n_neighbors = 5)
knn.fit(df_train, diag_train)

In [ ]:
knn.score(df_test, diag_test)

### We seem to get a pretty good prediction using the KNN Classifier. 
#### However I would like to improve the classification accuracy.
##### For this I would use something interesting:
##### First I would take the difference in means among all the scaled features. And then use these differences to remove extra features.

In [ ]:
dft = df_train.copy()
scaler = ss().fit(dft)
scaled_dft = pd.DataFrame(scaler.transform(dft))
scaled_dft.index = dft.index
scaled_dft.columns = dft.columns
scaled_dft['diagnosis'] = diag_train
scaled_dft.head()

In [ ]:
grouped_mean = scaled_dft.groupby(['diagnosis']).mean()

In [ ]:
diff_col = np.array(grouped_mean.loc['B']) - np.array(grouped_mean.loc['M'])
diff_col = np.absolute(diff_col)
diff_col = list(diff_col)

### I removed a lot of columns from the dataframe. Those will not be used for predictions.
#### These features did not show a good significant difference among the "M" and the "B" diagnosed patients.

In [ ]:
to_drop = []
columns = df.columns
for i in range(len(columns)):
    if diff_col[i] < 1:
        to_drop.append(columns[i])
    
dftr = df_train.copy()
dftr.drop(to_drop, axis = 1, inplace = True)

dfts = df_test.copy()
dfts.drop(to_drop, axis = 1, inplace = True)

In [ ]:
knn1 = KNN(n_neighbors = 5)
knn1.fit(dftr, diag_train)

In [ ]:
knn1.score(dfts, diag_test)

## It seems that the removal of all those columns didnot change the score of the prediction
#### So the removal of the not so different features was the right step

## Now I will automate a process which will run the KNN Classifier for different values of N and find the best among them

In [ ]:
for i in range(1, 20):
    knn = KNN(n_neighbors = i)
    knn.fit(dftr, diag_train)
    score = knn.score(dfts, diag_test)
    print("N = " + str(i) + " :: Score = " + str(score))

## It seems that after n = 9, the score remains fairly constant at 0.95.
### Also, at n = 5: score = 0.94.
Therefore, the optimal n is totally at the user's discretion.

Personally I would prefer n to be as low as possible. Since at n=5, the model gives pretty good accuracy, I would go with n = 5

## Now I would be looking at the actual probabilities of the prediction
### I will compute the prediction probabilities of the KNN classifier and check the statistics of the prediction

In [ ]:
knn = KNN(n_neighbors= 5)
knn.fit(dftr, diag_train)

In [ ]:
prediction_prob = knn.predict_proba(dfts)
predictions = knn.predict(dfts)

In [ ]:
from sklearn.metrics import confusion_matrix as cfm
cfm(predictions, diag_test)

## Tabulating the confusion matrix at different values of the cutoff

In [ ]:
def predict_cutoff(pred_prob, cutoff = 0.5):
    prediction_var = []
    for element in pred_prob:
        if element[0] > cutoff:
            prediction_var.append("B")
        else:
            prediction_var.append("M")
    return np.array(prediction_var)

In [ ]:
for i in range(1, 10):
    i = float(i) / 10
    print("Cutoff: " + str(i) + ";\n Confusion Matrix: ")
    print(cfm(predict_cutoff(prediction_prob, i), diag_test))
    print("\n\n")